In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import pickle
import re

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import scipy.stats as stats
from sklearn.model_selection import KFold
%matplotlib inline

In [2]:
with open('rookies3.pickle', 'rb') as to_read:
    df = pickle.load(to_read)

In [3]:
df.head()

,PLAYER,FIRST_YEAR,AST,BLK,DRB,EFG_PCT,FG2_PCT,FG2,FG2A,FG3_PCT,...,BPM,DRB_PCT,STL_PCT,TOV_PCT,TS_PCT,USG_PCT,DBPM,FTA_PER_FGA_PCT,AST_PCT,ORB_PCT
0,Aaron Brooks,2008,1.7,0.1,0.8,0.493,0.491,1.1,2.3,0.330,...,-1.8,7.9,1.1,15.1,0.535,21.8,-1.8,0.218,23.5,2.5
1,Aaron Gordon,2015,0.7,0.5,2.6,0.478,0.500,1.7,3.4,0.271,...,-2.8,17.8,1.3,13.9,0.517,15.5,0.0,0.293,6.3,6.4
2,Aaron Gray,2008,0.7,0.3,1.6,0.505,0.507,1.7,3.3,0.000,...,-2.0,18.7,1.8,18.7,0.529,22.0,0.4,0.485,11.4,12.5
3,Acie Law,2008,2.0,0.0,0.8,0.416,0.433,1.6,3.6,0.206,...,-6.7,6.0,1.8,17.8,0.455,16.5,-2.6,0.203,20.4,1.6
4,Al Horford,2008,1.5,0.9,6.6,0.499,0.502,4.1,8.2,0.000,...,-0.2,24.5,1.2,15.3,0.539,16.0,1.1,0.317,7.9,11.4


In [4]:
df.columns

Index(['PLAYER', 'FIRST_YEAR', 'AST', 'BLK', 'DRB', 'EFG_PCT', 'FG2_PCT',
       'FG2', 'FG2A', 'FG3_PCT', 'FG3', 'FG3A', 'FG_PCT', 'FG', 'FGA',
       'FT_PCT', 'FT', 'FTA', 'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV', 'TRB',
       'DRAFT_YEAR', 'ROUND', 'PICK_OVERALL', 'DRAFT_TEAM', 'AGE', 'POS',
       'BIRTH_COUNTRY', 'COLLEGE', 'FIFTH_YEAR', 'SALARY', 'G', 'GS', 'DWS',
       'WS', 'OWS', 'PER', 'BLK_PCT', 'VORP', 'TRB_PCT', 'FG3A_PER_FGA_PCT',
       'OBPM', 'WS_PER_48', 'BPM', 'DRB_PCT', 'STL_PCT', 'TOV_PCT', 'TS_PCT',
       'USG_PCT', 'DBPM', 'FTA_PER_FGA_PCT', 'AST_PCT', 'ORB_PCT'],
      dtype='object')

In [6]:
num = df[['SALARY','AST','BLK','DRB','EFG_PCT','FG2_PCT','FG2','FG2A','FG3_PCT','FG3','FG3A','FG_PCT','FG','FGA',
         'FT_PCT','FT','FTA','MP','ORB','PF','PTS','STL','TOV','TRB','PICK_OVERALL','G','GS','DWS','WS','OWS','PER',
         'BLK_PCT','VORP','TRB_PCT','FG3A_PER_FGA_PCT','OBPM','WS_PER_48','BPM','DRB_PCT','STL_PCT','TOV_PCT',
         'TS_PCT','USG_PCT','DBPM','FTA_PER_FGA_PCT','AST_PCT','ORB_PCT']]

In [8]:
X = num.iloc[:,1:]
y = num.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

lm = LinearRegression()

lm.fit(X_train, y_train)
lm.score(X_train, y_train)

0.37517792616144907

In [13]:
five = num[['SALARY','PTS','AST','TRB','BLK','STL']]

X = five.iloc[:,1:]
y = five.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

lm = LinearRegression()

lm.fit(X_train, y_train)
lm.score(X_train, y_train)

0.19111882964937366

In [16]:
df.columns

Index(['PLAYER', 'FIRST_YEAR', 'AST', 'BLK', 'DRB', 'EFG_PCT', 'FG2_PCT',
       'FG2', 'FG2A', 'FG3_PCT', 'FG3', 'FG3A', 'FG_PCT', 'FG', 'FGA',
       'FT_PCT', 'FT', 'FTA', 'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV', 'TRB',
       'DRAFT_YEAR', 'ROUND', 'PICK_OVERALL', 'DRAFT_TEAM', 'AGE', 'POS',
       'BIRTH_COUNTRY', 'COLLEGE', 'FIFTH_YEAR', 'SALARY', 'G', 'GS', 'DWS',
       'WS', 'OWS', 'PER', 'BLK_PCT', 'VORP', 'TRB_PCT', 'FG3A_PER_FGA_PCT',
       'OBPM', 'WS_PER_48', 'BPM', 'DRB_PCT', 'STL_PCT', 'TOV_PCT', 'TS_PCT',
       'USG_PCT', 'DBPM', 'FTA_PER_FGA_PCT', 'AST_PCT', 'ORB_PCT'],
      dtype='object')

In [17]:
gamestats = num[['SALARY','G','GS','MP','FG','FGA','FG_PCT','FG3','FG3A','FG3_PCT','FG2','FG2A','FG2_PCT','EFG_PCT',
                'FT','FTA','FT_PCT','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']]

In [20]:
X = gamestats.iloc[:,1:]
y = gamestats.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

lm = LinearRegression()

lm.fit(X_train, y_train)
lm.score(X_train, y_train)

0.2699683653195747

In [27]:
lasso = Lasso(alpha=1000)

scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

lasso.fit(X_train_sc, y_train)
lasso.score(X_test_sc, y_test)

/Users/Stephen/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


-0.004741719195449301